# Using Skeel-Berzins integrator to solve for the heat equation

In [ ]:
using Plots
using ForwardDiff
using SkeelBerzins1
using DifferentialEquations

## The heat equation in standard formulation

$$
\partial_t u = D \partial_x^2 u \qquad \text{on} \quad \Omega \in [0,1]
$$
For the sake of simplicity, let $D=1$. In flux form, we have
$$
\partial_t u = (\partial_x u)_x =: f(x, t, u, \partial_x u)_x
$$

In [ ]:
# Define the mesh

N_x = 21
L = 1
T = 1

x_mesh = collect(LinRange(0, L, N_x));

The differential equation in the form

$$
c(x,t,u, \partial_x u) \partial_t u = f(x, t, u, \partial_x u)_x + s(x, t, u)
$$

Here, we have

$$
m = 0 \\
c = 1 \\
f = \partial_x u \\
s = 0
$$

In [ ]:
function diff_eq(x, t, u, dudx)

    c = 0u + 1
    f = dudx
    s = 0u

    return c, f, s
end;

Boundary conditions may be given through the expression

$$
p(x, t, u) + q(x, t) f (x, t, u, \partial_x u) = 0
$$

for $x = 0, 1\,$.
E.g., $u(x=0) = 1$ and $\partial_x u(x=1) = f(x, t, u, \partial_x u) =-2$ translates into

$$
p_L = u_L - 1 \quad \text{and}\quad q_L = 0 \\
p_R = 2 \quad\text{and}\quad q_R = 1
$$

In [ ]:
function bdfun(xl, ul, xr, ur, t)

    pl = ul - 1
    ql = 0

    pr = 2
    qr = 1

    return pl, ql, pr, qr
end;

Set Gaussian initial data.

In [ ]:
icfun(x) = exp(-100 * x^2);

In [ ]:
# Solve by the explicit Euler method implemented into SkeelBerzins.jl.

params_diffEq = SkeelBerzins.Params(; solver=:DiffEq)

t_span = (0, 1.0)
m = 0

sol_euler = pdepe(m, diff_eq, icfun, bdfun, x_mesh, t_span);

In [ ]:
# Use DifferentialEquations to solve the problem.

pb = pdepe(m, diff_eq, icfun, bdfun, x_mesh, t_span; params=params_diffEq)

problem = DifferentialEquations.ODEProblem(pb)
sol_diffEq = DifferentialEquations.solve(problem, Rosenbrock23())
sol_reshaped_diffEq = reshape(sol_diffEq, pb);

In [ ]:
plot(x_mesh, sol_euler[end])
plot!(x_mesh, sol_diffEq.u[end])

### Mixed formulation of the diffusion equation

For testing the implementation of a higher-than-second-order-in-space equation, artificially inflate the equation to a system of two equations.

$$
\partial_t u = -\mu \\
0 = - \partial^2_x u - \mu
$$

We define $\psi:=(u, \mu)$. Hence, for the standard form $c(x,t,\psi, \partial_x \psi) \partial_t \psi = f(x, t, \psi, \partial_x \psi)_x + s(x, t, \psi)$, we have

$$
c(x,t,\psi,\partial_x \psi) \partial_t \psi = (\partial_t u, 0)^T \\
f(x, t, \psi, \partial_x \psi) = (0, -\partial_x u)^T \\
s(x, t, u) = (-\mu, -\mu)^T
$$

In [ ]:
function diff_eq(x, t, u, dudx)

    c = SVector(1, 0)
    f = SVector(0, -dudx[1])
    s = SVector(-u[2], -u[2])

    return c, f, s
end;

Boundary conditions $u(x=0) = 1$ and $\partial_x u(x=1) = -f(x, t, \psi, \partial_x \psi)_2 =-2$ translate into

$$
p_{L,1} = u_L - 1 \quad\text{and}\quad q_L = 0 \\
p_{R,2} = 2 \quad\text{and}\quad q_{R,2} = 1 
$$

In [ ]:
function bdfun(xl, ul, xr, ur, t)

    pl = SVector(ul[1] - 1.0, ul[1] - 1.0)
    ql = SVector(0., 0.)

    pr = SVector(0, 2)
    qr = SVector(0, 1)

    return pl, ql, pr, qr
end;

In [ ]:
icfun(x) = SVector(exp(-100 * x^2), 0 * x);

In [ ]:
params_diffEq = SkeelBerzins.Params(; solver=:DiffEq)

pb = pdepe(m, diff_eq, icfun, bdfun, x_mesh, t_span; params=params_diffEq)
problem = DifferentialEquations.ODEProblem(pb)
sol_diffEq = DifferentialEquations.solve(problem, Rosenbrock23())
sol_reshaped_diffEq = reshape(sol_diffEq, pb)